In [ ]:
import pandas as pd
from unsloth import FastLanguageModel
from utils import inference
import os

In [ ]:
model_name = "models/fine_tuned/Llama-3.2-3B-Instruct"

In [ ]:
def list_folders_os(directory_path):
    try:
        # List all entries in the directory
        entries = os.listdir(directory_path)
        # Filter out directories
        folders = [entry for entry in entries if os.path.isdir(os.path.join(directory_path, entry))]
        return folders
    except FileNotFoundError:
        print(f"The directory {directory_path} does not exist.")
        return []
    except PermissionError:
        print(f"Permission denied for accessing {directory_path}.")
        return []

# Usage
directory = "models/fine_tuned/"
folders = list_folders_os(directory)
print("Folders:", folders)

In [ ]:
origin = model_name.split("/", 1)[0]
if origin == "unsloth":
    output_folder = f"results/vanilla/{model_name.split('/', 1)[1]}"
elif origin == "fine_tuned":
    output_folder = f"results/fine_tuned/{model_name.split('/', 1)[1]}"
os.makedirs(output_folder, exist_ok=True)

In [ ]:
max_seq_length = 16384 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False
"\n",
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name, # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference"

In [ ]:
df_Breast = pd.read_csv('../data/gt_processed/Breast_gs_processed.csv')
df_GI = pd.read_csv('../data/gt_processed/GI_gs_processed.csv')
df_Neuro = pd.read_csv('../data/gt_processed/Neuro_gs_processed.csv')

In [ ]:
inference(df_Breast, tokenizer, model, f"{output_folder}/Breast_output.csv")
inference(df_GI, tokenizer, model, f"{output_folder}/GI_output.csv")
inference(df_Neuro, tokenizer, model, f"{output_folder}/Neuro_output.csv")